In [ ]:
from sklearn.datasets import load_diabetes

from ipyml.api import RegressionBase

In [ ]:
diabetes = load_diabetes(return_X_y=False, as_frame=True)["frame"]

In [ ]:
regr = RegressionBase(diabetes)

In [ ]:
regr

In [ ]:
regr.target_select.options = sorted(regr.target_select.options)

In [ ]:
import json
import os
import time
from pathlib import Path

IS_TESTING = json.loads(os.environ.get("IPYML_TESTING", "false"))

if IS_TESTING:
    # automate the "user interaction" of the tool
    target_feature = "target"
    input_features = ["age", "sex", "bmi", "bp", "s1"]
    validation_column = None
    test_model_filename = "test_model"

    assert (
        target_feature in regr.target_select.options
    ), "Failed to find the target feature 'target'."
    regr.target_select.value = "target"
    assert all(
        [_input in regr.inputs_select.options for _input in input_features]
    ), "Failed to find all input features."
    regr.inputs_select.value = input_features
    assert validation_column in regr.validation_column_select.options
    regr.validation_column_select.value = validation_column

    # No validation - linear regression
    regr.lr_button.click()

    # Get latest regression model widget and run
    model_widget = regr._generated_models[-1]
    model_widget.run_button.click()
    # give it a few seconds to build the model
    time.sleep(3)

    trained_model_widget = model_widget._trained_models[-1]
    trained_model_widget.model_name.value = test_model_filename + "_lr"
    trained_model_widget.export_button.click()
    # give it a few seconds to save the model
    time.sleep(5)

    assert Path(f"{test_model_filename}_lr.json").is_file()

In [ ]:
# imports
import json
from functools import partial

from ipyml.regression_callable import run_linear_regression, run_neural_net

In [ ]:
# load the model
if Path("test_model_lr.json").is_file():
    model_name = "test_model_lr.json"
    reg_call = run_linear_regression
elif Path("test_model_nn.json").is_file():
    model_name = "test_model_nn.json"
    reg_call = run_neural_net
else:
    raise NotImplementedError

with open(model_name, "r") as f:
    model_data = json.load(f)

In [ ]:
# setup a partially initialized function
partial_func = partial(reg_call, model=model_data)

# use it to make predictions
inputs = {var: diabetes[var] for var in diabetes.columns}
predictions = partial_func(**inputs)
print(predictions)